# Welche Entwickler haben die meisten Commits?

## 1. Verbindung zur Datenbank
Es wird eine Verbindung zur Neo4j-Datenbank aufgebaut.

In [12]:
import py2neo

graph = py2neo.Graph(bolt=True, host='localhost', user='neo4j', password='neo4j')

## 2. Cypher-Abfrage
Es wird eine Abfrage an die Datenbank gestellt. Das Ergebnis wird in einem Dataframe (pandas) gespeichert.

In [13]:
import pandas as pd

query ="MATCH (a:Author)-[:COMMITTED]->(c:Commit)-[:CONTAINS_CHANGE]->(:Change)-[:MODIFIES]->(file:File) WHERE NOT c:Merge RETURN a.name as developer, count(distinct c) as commits"
df = pd.DataFrame(graph.run(query).data())


## 3. Datenaufbereitung
Zur Kontrolle werden die ersten fünf Zeilen des Ergebnisses der Abfrage als Tabelle ausgegeben.

In [14]:
df.head()

,commits,developer
0,1,Joseph Walton
1,23,Noel Yap
2,2,Robin Stevens
3,1,Chris Barrow
4,1,Arjen Stolk


Im folgenden Codeabschnitt wird das Dataframe absteigend entsprechend der der Anzahl der Commits (commits) sortiert und die ersten zehn Entwickler (developer) extrahiert. Die Werte werden in den Variablen <font face="Courier">x_commits</font> und <font face="Courier">y_developers</font> gespeichert.

In [15]:
import numpy as np
# Sortiere Zeilen des Dataframes aufsteigend ihrer Anzahl der Commits in neuem Dataframe.
sorted_commits=df.sort_values('commits',ascending=False)
# Extrahiere die ersten 10 Zeilen dieses Dataframes in neuem Dataframe.
top10_commits=sorted_commits.head(10)

# Hole Werte aus Spalten aus Dataframe. Umwandlung dieser Werte in list Objekt.
x_commits=top10_commits['commits'].tolist()
y_developers=top10_commits['developer'].tolist()

x_commits.reverse()
y_developers.reverse()

# 4. Visualisierung
Die Daten werden mittels eines Bar Charts visualisiert (https://plot.ly/python/horizontal-bar-charts/, x=x_commits, y=y_developer).

In [16]:
from IPython.display import display, HTML

base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

In [17]:
import pygal 
bar_chart = pygal.HorizontalBar()
bar_chart.title = 'Entwickler mit den meisten Commits'
bar_chart.x_title='Anzahl der Commits'
bar_chart.y_title='Entwickler'
bar_chart.show_legend=False
bar_chart.x_labels= y_developers
bar_chart.add(' ',x_commits)
display(HTML(base_html.format(rendered_chart=bar_chart.render(is_unicode=True))))